<a href="https://colab.research.google.com/github/Pamela2710/Data-Mining/blob/main/TextMining/TextMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [105]:
#df = pd.read_csv("sample_data/livetweets_data_no_json.csv",encoding='latin1', skipinitialspace=True)
#df.head()
#pd.read_csv("sample_data/livetweets_data_no_json.csv",encoding='latin1', skipinitialspace=True)

In [ ]:
df = pd.read_csv("sample_data/livetweets_data_no_json.csv")
#df.head()

In [ ]:
df

In [108]:
df_candidatos = pd.read_csv("sample_data/account_info.csv")

In [109]:
df_tweets = df.fillna("")

In [110]:
#----------------------------------------LITERAL A--------------------------------------
# Filtra la tabla de candidatos solo para aquellos cuyos twitter_screen_name aparecen en la tabla de tweets
candidatos_filtrados = df_tweets[df_tweets['tweet_screen_name'].isin(df_candidatos['twitter_screen_name'])]
# Obtén la frecuencia de cada tweet_screen_name en la tabla de tweets

frecuencia_tweet_screen_name = candidatos_filtrados['tweet_screen_name'].value_counts()

# Muestra el resultado
top_10 = frecuencia_tweet_screen_name.head(10)
top_10


HernanUlloa        1183
abenavidesgol1     1070
wgomezr             804
jimmyjairala        662
LuisaMaldonadoM     574
LoroHomero          509
VickyDesintonio     477
CynthiaViteri6      357
fcojimenez21        335
davidroserow        305
Name: tweet_screen_name, dtype: int64

In [111]:
#Tabla del Top 10
# Filtra la tabla de candidatos_filtrados solo para aquellos en el top 10
top_10_candidatos_filtrados = candidatos_filtrados[candidatos_filtrados['tweet_screen_name'].isin(top_10.index)]

top_10_candidatos_filtrados

top_10_candidatos_filtrados2 = candidatos_filtrados[candidatos_filtrados['tweet_screen_name'].isin(top_10.index)]

top_10_candidatos_filtrados2

,tweet_id,tweet_date,tweet_screen_name,tweet_text
21,1096984892318015489,2019-02-17,abenavidesgol1,Énner Valencia estuvo presente en las redes co...
152,1097006152498532353,2019-02-17,abenavidesgol1,Jairo Vélez cumplió destacada actuación en tri...
158,1097008003398672384,2019-02-17,LuisaMaldonadoM,Desde cada uno de los rincones de #Quito los q...
195,1097012227855564800,2019-02-17,abenavidesgol1,Pocos minutos en cancha estuvo Fernando Gaibor...
1157,1097162552348418050,2019-02-17,LoroHomero,Disfrutamos una linda noche en Barrio #ElConde...
...,...,...,...,...
1198815,1129607787313602561,2019-05-18,abenavidesgol1,Ecuador cuenta con primera lista de 40 jugador...
1198827,1129608232568283137,2019-05-18,VickyDesintonio,@VickyDesintonio Victoria de la Patria ✊🇪🇨
1198836,1129609424102936576,2019-05-18,abenavidesgol1,Vuelve a la senda del triunfo Independiente de...
1198884,1129613355231191040,2019-05-18,VickyDesintonio,@VickyDesintonio ha hecho un ejercicio de resi...


In [141]:
agrupado_por_usuario = top_10_candidatos_filtrados.groupby('tweet_screen_name')['tweet_text'].apply(' '.join).reset_index(name='tweets_agrupados')
agrupado_por_usuario

,tweet_screen_name,tweets_agrupados
0,CynthiaViteri6,¡Buenos días guayaquileños! Ya estamos en vivo...
1,HernanUlloa,@HernanUlloa abogado se ganó mi respeto y admi...
2,LoroHomero,Disfrutamos una linda noche en Barrio #ElConde...
3,LuisaMaldonadoM,Desde cada uno de los rincones de #Quito los q...
4,VickyDesintonio,#ManabíEsVictoria\nAmigas y amigos deseo para ...
5,abenavidesgol1,Énner Valencia estuvo presente en las redes co...
6,davidroserow,"Martes 08:00 en Quito en el @CongopeEcuador, d..."
7,fcojimenez21,¡Seguimos recorriendo Guayaquil! Estamos en Me...
8,jimmyjairala,Se ve que los adversarios políticos de @jimmyj...
9,wgomezr,Comparto con ustedes mí Agenda de actividades ...


In [142]:
# Preprocesamiento de texto

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('spanish'))  # Obtener stopwords en español
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenización y conversión a minúsculas
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]  # Eliminar stopwords y signos de puntuación
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]  # Lematización
    #processed_text = ' '.join(lemmatized_tokens)  # Unir los tokens en una cadena de texto
    #print(lemmatized_tokens)
    return lemmatized_tokens

# Aplicar preprocesamiento de texto a los tweets
agrupado_por_usuario['preprocessed_text'] = agrupado_por_usuario['tweets_agrupados'].apply(preprocess_text)

In [164]:
#--------------------------------LITERAL B-----------------------------------
from collections import Counter

# Crear un diccionario para almacenar las frecuencias
frecuencias_por_usuario = {}
frecuencias_palabras = []
# Iterar sobre cada fila del DataFrame
for index, row in agrupado_por_usuario.iterrows():
    tweet_screen_name = row['tweet_screen_name']
    preprocessed_text = row['preprocessed_text']

    # Contar las frecuencias de palabras en el texto preprocesado
    frecuencias_palabras = Counter(preprocessed_text).most_common(10)
    #print(frecuencias_palabras)
    print(frecuencias_palabras)
    # Almacenar las frecuencias en el diccionario, utilizando tweet_screen_name como clave
    frecuencias_por_usuario.setdefault(tweet_screen_name, []).append(frecuencias_palabras)
#print(frecuencias_por_usuario)
# Crear un DataFrame a partir del diccionario de frecuencias

df_frecuencias = pd.DataFrame(frecuencias_por_usuario).transpose().fillna(0)
df_frecuencias


[('http', 410), ('wgomezr', 389), ('cpccs', 281), ('cnegobec', 233), ('vickydesintonio', 207), ('pueblo', 167), ('q', 123), ('ecuador', 106), ('x', 96), ('kevincasasz', 91)]


,0
CynthiaViteri6,"[(http, 362), (guayaquil, 153), (ciudad, 84), ..."
HernanUlloa,"[(hernanulloa, 918), (http, 740), (cpccs, 547)..."
LoroHomero,"[(lorohomero, 438), (http, 388), (quito, 157),..."
LuisaMaldonadoM,"[(http, 500), (quito, 314), (compromisorc5, 17..."
VickyDesintonio,"[(http, 396), (victoriadelapatria, 226), (vick..."
abenavidesgol1,"[(http, 1976), (triunfo, 226), (abenavidesgol1..."
davidroserow,"[(http, 251), (davidroserow, 155), (cpccs, 146..."
fcojimenez21,"[(http, 364), (jiménezalcalde, 158), (elguayaq..."
jimmyjairala,"[(http, 673), (elmejorguayaquildetodoslostiemp..."
wgomezr,"[(http, 410), (wgomezr, 389), (cpccs, 281), (c..."


In [132]:
import math
frecuencias_por_candidato = []
# Paso 4: Concatenar todos los tweets de los top 10 candidatos en una sola cadena
texto_campaña = ' '.join(top_10_candidatos_filtrados['tweet_text'])
print('len ', len(top_10_candidatos_filtrados2))
# Paso 5: Preprocesar el texto (tokenizar, eliminar stopwords, lematizar, etc.)
# (Asumiendo que tienes una función preprocess_text() que hace esto)
texto_preprocesado = preprocess_text(texto_campaña)
print('texto_preprocesado ',texto_preprocesado)
print('text_procesado_len ', len(texto_preprocesado))
 # Paso 6: Contar la frecuencia de palabras en el texto preprocesado
frecuencias_palabras = Counter(texto_preprocesado)
print('frecuencia ',frecuencias_palabras)
# Paso 7: Seleccionar las 10 palabras más comunes
top_10_palabras = frecuencias_palabras.most_common(10)
print('top 10 ',top_10_palabras)
print("\nTop 10 palabras más utilizadas en campaña por los top 10 candidatos:")
# Paso 8: Calcular los índices de términos: tf, df, idf, tf-idf # Calcular TF para cada palabra
# Agregar el nombre del candidato y las frecuencias de palabras a la lista
for index, row in agrupado_por_usuario.iterrows():
    # Obtener los tweets agrupados y el nombre del candidato
    tweets_agrupados = row['tweets_agrupados']
    candidato = row['tweet_screen_name']
    frecuencias_por_candidato.append({'Candidato': candidato, 'Frecuencias': top_10_palabras})

tf = []
for i, candidato_info in enumerate(frecuencias_por_candidato):
    tf.append({palabra: frecuencia / long_euclidiana[i] for palabra, frecuencia in candidato_info['Frecuencias']})

print('tf', tf)

# Convertir la lista de diccionarios 'tf' en un DataFrame
df_tf = pd.DataFrame(tf)

# Cambiar los nombres de las columnas por los nombres de los candidatos
#df_tf.row = [candidato_info['Candidato'] for candidato_info in frecuencias_por_candidato]

# Transponer el DataFrame para que las filas representen palabras y las columnas representen candidatos
df_tf = df_tf.transpose()
df_tf.columns = [candidato_info['Candidato'] for candidato_info in frecuencias_por_candidato]
# Imprimir el DataFrame resultante
df_tf['df'] = df_tf.apply(lambda row: sum(1 for value in row if value != 0), axis=1)
df_tf['idf'] = abs(np.log(df_tf['df'] / 10))
df_tf


len  6276
texto_preprocesado  ['énner', 'valencia', 'presente', 'redes', 'contrarias', 'triunfo', 'tigres', 'necaxa', 'http', 'http', 'jairo', 'vélez', 'cumplió', 'destacada', 'actuación', 'triunfo', 'atlante', 'dorado', 'http', 'http', 'cada', 'rincones', 'quito', 'quiteños', 'quiteñas', 'muestran', 'apoyo', 'cariño', 'militante', 'revolución', 'ciudadana', 'luisamaldonadom', 'sur', 'lista', '5', 'compromiso', 'social', 'arecuperarlapatria', 'todotodito5', 'http', 'pocos', 'minutos', 'cancha', 'fernando', 'gaibor', 'empate', 'independiente', 'avellaneda', 'http', 'http', 'disfrutamos', 'linda', 'noche', 'barrio', 'elconde4', 'gracias', 'recibimiento', 'tan', 'emotivo', 'permitirnos', 'compartir', 'plan', 'trabajo', 'lorohomero', 'futuro', 'alcalde', 'quito', 'votatodo19', 'quitograndeotravez', 'http', 'negado', 'recurso', 'apelación', 'propuesto', 'barcelona', 'nivel', 'conmebol', 'http', 'http', 'felipe', 'caicedo', 'actuó', 'cambio', 'derrota', 'sufrida', 'lazio', 'http', 'http', 'i

,CynthiaViteri6,HernanUlloa,LoroHomero,LuisaMaldonadoM,VickyDesintonio,abenavidesgol1,davidroserow,fcojimenez21,jimmyjairala,wgomezr,df,idf
http,12.844836,3.902278,3.587466,3.275446,3.120069,2.155171,2.136078,2.103848,2.024912,1.949578,10,0.0
cpccs,2.291298,0.696099,0.639942,0.584283,0.556567,0.384446,0.381040,0.375290,0.361210,0.347771,10,0.0
hernanulloa,1.994553,0.605948,0.557064,0.508613,0.484486,0.334656,0.331691,0.326687,0.314429,0.302731,10,0.0
quito,1.212417,0.368334,0.338619,0.309167,0.294502,0.203425,0.201623,0.198581,0.191130,0.184020,10,0.0
voto,1.104317,0.335493,0.308427,0.281602,0.268244,0.185288,0.183646,0.180875,0.174089,0.167612,10,0.0
guayaquil,1.036489,0.314887,0.289484,0.264306,0.251768,0.173907,0.172367,0.169766,0.163396,0.157317,10,0.0
lorohomero,0.938987,0.285266,0.262252,0.239443,0.228084,0.157548,0.156152,0.153796,0.148026,0.142519,10,0.0
cnegobec,0.938987,0.285266,0.262252,0.239443,0.228084,0.157548,0.156152,0.153796,0.148026,0.142519,10,0.0
wgomezr,0.911432,0.276894,0.254556,0.232416,0.221391,0.152925,0.151570,0.149283,0.143682,0.138336,10,0.0
pueblo,0.849964,0.258220,0.237388,0.216742,0.206460,0.142611,0.141348,0.139215,0.133992,0.129007,10,0.0


In [133]:
from math import sqrt
import math
import numpy as np

# Crear una lista para almacenar las frecuencias de palabras por candidato
frecuencias_por_candidato = []
fre_cuadrado = 0
fre_sqrt = 0
long_euclidiana = []
# Iterar sobre cada fila del DataFrame
for index, row in agrupado_por_usuario.iterrows():
    # Obtener los tweets agrupados y el nombre del candidato
    tweets_agrupados = row['tweets_agrupados']
    candidato = row['tweet_screen_name']

    # Preprocesar los tweets agrupados
    texto_preprocesado = preprocess_text(tweets_agrupados)

    # Contar la frecuencia de palabras en el texto preprocesado
    frecuencias_palabras = Counter(texto_preprocesado)
    #print('values',frecuencias_palabras.values())

    for i in frecuencias_palabras.values():
      fre_cuadrado+=pow(i,2)
    sqrt(fre_cuadrado)
    long_euclidiana.append(sqrt(fre_cuadrado))

    # Obtener las 10 palabras más frecuentes
    palabras_mas_frecuentes = frecuencias_palabras.most_common(10)

    # Agregar el nombre del candidato y las frecuencias de palabras a la lista
    frecuencias_por_candidato.append({'Candidato': candidato, 'Frecuencias': palabras_mas_frecuentes})

# Imprimir las frecuencias de palabras por candidato
#print("\nTop 10 palabras más utilizadas en campaña por los top 10 candidatos:")
#for candidato_info in frecuencias_por_candidato:
    #print("Candidato:", candidato_info['Candidato'])
    #print("Frecuencia de palabras:", candidato_info['Frecuencias'])
    #print()

# Calcular los índices de términos: tf, df, idf, tf-idf
# Calcular TF para cada palabra
tf = []

for i, candidato_info in enumerate(frecuencias_por_candidato):
    tf.append({palabra: frecuencia / long_euclidiana[i] for palabra, frecuencia in candidato_info['Frecuencias']})

#print('tf', tf)
# Convertir la lista de diccionarios 'tf' en un DataFrame
df_tf = pd.DataFrame(tf)

# Cambiar los nombres de las columnas por los nombres de los candidatos
#df_tf.row = [candidato_info['Candidato'] for candidato_info in frecuencias_por_candidato]

# Transponer el DataFrame para que las filas representen palabras y las columnas representen candidatos
df_tf = df_tf.transpose()
df_tf.columns = [candidato_info['Candidato'] for candidato_info in frecuencias_por_candidato]


# Imprimir el DataFrame resultante
df_tf.fillna(0, inplace=True)
df_tf['df'] = df_tf.apply(lambda row: sum(1 for value in row if value != 0), axis=1)
df_tf['idf'] = np.log(10/ df_tf['df'] )
for column in df_tf.columns[:-2]:
    df_tf[column + '(tf-idf)'] = df_tf[column] * df_tf['idf']

df_tf

,CynthiaViteri6,HernanUlloa,LoroHomero,LuisaMaldonadoM,VickyDesintonio,abenavidesgol1,davidroserow,fcojimenez21,jimmyjairala,wgomezr,...,CynthiaViteri6(tf-idf),HernanUlloa(tf-idf),LoroHomero(tf-idf),LuisaMaldonadoM(tf-idf),VickyDesintonio(tf-idf),abenavidesgol1(tf-idf),davidroserow(tf-idf),fcojimenez21(tf-idf),jimmyjairala(tf-idf),wgomezr(tf-idf)
http,0.767299,0.476516,0.229693,0.270251,0.203886,0.702742,0.088475,0.126370,0.224879,0.131902,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
guayaquil,0.324300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034023,0.051124,0.000000,...,0.390449,0.0,0.000000,0.000000,0.0,0.0,0.0,0.040962,0.061552,0.000000
ciudad,0.178047,0.000000,0.032560,0.047024,0.000000,0.000000,0.000000,0.027079,0.019714,0.000000,...,0.123413,0.0,0.022569,0.032594,0.0,0.0,0.0,0.018770,0.013665,0.000000
sigueguayaquilsigue,0.131416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.302596,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
progreso,0.120818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.278193,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rtw24ec,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019046,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.043856,0.000000
wgomezr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125146,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.288160
q,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039571,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.091115
x,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030884,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.071114


In [134]:
#--------------------------------------LITERAL C------------------------------------

agrupado_por_usuario = top_10_candidatos_filtrados.groupby(['tweet_screen_name','tweet_date'])['tweet_text'].apply(' '.join).reset_index(name='tweets_agrupados')

#agrupado_por_usuario

# Convertir la columna 'tweet_date' a tipo datetime
agrupado_por_usuario['tweet_date'] = pd.to_datetime(agrupado_por_usuario['tweet_date'])

# Agrupar por candidato y semana, y concatenar los tweets
agrupado_por_semana = (agrupado_por_usuario
                       .groupby(['tweet_screen_name', pd.Grouper(key='tweet_date', freq='W')])
                       .agg({'tweets_agrupados': ' '.join})
                       .reset_index())

agrupado_por_semana

,tweet_screen_name,tweet_date,tweets_agrupados
0,CynthiaViteri6,2019-02-24,¡Buenos días guayaquileños! Ya estamos en vivo...
1,CynthiaViteri6,2019-03-03,Mañana arrancamos la semana con agenda de medi...
2,CynthiaViteri6,2019-03-10,"Cada semana que pasa, nos acerca más a ese #Gu..."
3,CynthiaViteri6,2019-03-17,Gracias Guayaquil por demostrar una vez más qu...
4,CynthiaViteri6,2019-03-24,¡Buenos días guayaquileños! Empezamos esta sem...
...,...,...,...
125,wgomezr,2019-04-21,Para tod@s mis amig@s y seguidores #FelizLunes...
126,wgomezr,2019-04-28,Esta bien ganastes lol X q el dolor ? Jajaja t...
127,wgomezr,2019-05-05,Feliz 1 de mayo DIA DEL TRABAJADOR #CPCCS #Wal...
128,wgomezr,2019-05-12,#FelizMartes #07May #Corrupcion #Destruccion @...


In [ ]:
from collections import Counter

# Aplicar preprocesamiento de texto a los tweets
agrupado_por_semana['preprocessed_text'] = agrupado_por_semana['tweets_agrupados'].apply(preprocess_text)
# Crear un diccionario para almacenar las frecuencias


frecuencias_palabras2 = []
frecuencia_global = []

# Iterar sobre cada fila del DataFrame
for index, row in agrupado_por_semana.iterrows():
    tweet_screen_name = row['tweet_screen_name']
    preprocessed_text = row['preprocessed_text']

    # Contar las frecuencias de palabras en el texto preprocesado
    frecuencias_palabras = Counter(preprocessed_text)
    frecuencias_palabras2.append(frecuencias_palabras)

agrupado_por_semana['Frecuencia'] = frecuencias_palabras2


frecuencias_verificadas = []

# Iterar sobre cada fila del DataFrame
for index, row in agrupado_por_semana.iterrows():
    # Obtener las frecuencias de palabras de la fila actual
    frecuencias_palabras2 = row['Frecuencia']
    print('FP',frecuencias_palabras2.items())
    #print(frecuencias_por_usuario)
    print(frecuencias_palabras)
    # Verificar si cada palabra está presente en frecuencias_palabras
    frecuencias_verificadas_usuario = {}
    for palabra, frecuencia in frecuencias_por_usuario.items():
        if palabra in frecuencias_por_usuario:
            frecuencias_verificadas_usuario[palabra] = frecuencia

    # Agregar el diccionario verificado a la lista
    frecuencias_verificadas.append(frecuencias_verificadas_usuario)

# Agregar la lista de diccionarios verificados como una nueva columna al DataFrame
agrupado_por_semana['Frecuencia_verificada'] = frecuencias_verificadas


#agrupado_por_semana['Frecuencia_global'] = frecuencia_global

print('FRU',frecuencias_palabras2)
#print(frecuencias_por_usuario)
# Crear un DataFrame a partir del diccionario de frecuencias
#df_frecuencias = pd.DataFrame(frecuencias_por_usuario).transpose().fillna(0)

agrupado_por_semana

In [ ]:
#--------------------------------LITERAL D-----------------------------------
query = input("Ingresa un query: ")

#Separar cada palabra en un arreglo
palabras = query.split()

#Imprimir el arreglo de palabras
print("Arreglo de palabras:", palabras)

In [ ]:
#--------------------------------LITERAL E-----------------------------------
#Obtener la lista de columnas disponibles
columnas_disponibles = df_tf.columns[:10].tolist()

#Mostrar al usuario las opciones disponibles
print("Candidatos disponibles:")
for i, columna in enumerate(columnas_disponibles, 1):
    print(f"{i}. {columna}")

#Solicitar al usuario que elija la primera columna
opcion1 = int(input("Seleccione el primer candidat@ (ingrese el número): ")) - 1

#Solicitar al usuario que elija la segunda columna
opcion2 = int(input("Seleccione el segunda candidat@ (ingrese el número): ")) - 1

#Verificar si las opciones ingresadas son válidas
if opcion1 not in range(len(columnas_disponibles)) or opcion2 not in range(len(columnas_disponibles)):
    print("Error: Al menos una de las opciones ingresadas no es válida.")
else:
    nombre_columna1 = columnas_disponibles[opcion1]
    nombre_columna2 = columnas_disponibles[opcion2]

    nueva_columna = df_tf[nombre_columna1] * df_tf[nombre_columna2]

    # Crea un nuevo DataFrame con la nueva columna
    tabla_similaridad = pd.DataFrame({f"Similaridad": nueva_columna})
    print(tabla_similaridad)